In [41]:
!nvidia-smi

Wed Mar 12 10:57:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             36W /  250W |    1143MiB /  16384MiB |     30%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [42]:
!pip install gdown

In [43]:
! rm -rf video.mp4

In [44]:
import gdown

# Update with your file's specific ID
file_id = "1Xf3sROm-sbl9GN33jBOJJdqCxEafrQ4J"
url = f"https://drive.google.com/uc?id={file_id}"

output = "people-counting.mp4"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1Xf3sROm-sbl9GN33jBOJJdqCxEafrQ4J
To: /kaggle/working/people-counting.mp4
100%|██████████| 2.55M/2.55M [00:00<00:00, 164MB/s]


'people-counting.mp4'

In [45]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [46]:
SOURCE_VIDEO_PATH = "/kaggle/working/people-counting.mp4"

In [47]:
# Pip install method (recommended)

!pip install "ultralytics<=8.3.40"

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6170.1/8062.4 GB disk)


In [48]:
!pip install supervision==0.3.0

from IPython import display
display.clear_output()

import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.3.0


In [49]:
# settings
MODEL = "yolov8x.pt"

In [50]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


In [51]:
from ultralytics import YOLO

# Load YOLO model
model = YOLO('yolov8s.pt')

# Get class names from the model
class_names = model.names  # Dictionary {class_id: class_name}

# Print all class IDs and names
for class_id, class_name in class_names.items():
    print(f"Class ID: {class_id}, Class Name: {class_name}")

Class ID: 0, Class Name: person
Class ID: 1, Class Name: bicycle
Class ID: 2, Class Name: car
Class ID: 3, Class Name: motorcycle
Class ID: 4, Class Name: airplane
Class ID: 5, Class Name: bus
Class ID: 6, Class Name: train
Class ID: 7, Class Name: truck
Class ID: 8, Class Name: boat
Class ID: 9, Class Name: traffic light
Class ID: 10, Class Name: fire hydrant
Class ID: 11, Class Name: stop sign
Class ID: 12, Class Name: parking meter
Class ID: 13, Class Name: bench
Class ID: 14, Class Name: bird
Class ID: 15, Class Name: cat
Class ID: 16, Class Name: dog
Class ID: 17, Class Name: horse
Class ID: 18, Class Name: sheep
Class ID: 19, Class Name: cow
Class ID: 20, Class Name: elephant
Class ID: 21, Class Name: bear
Class ID: 22, Class Name: zebra
Class ID: 23, Class Name: giraffe
Class ID: 24, Class Name: backpack
Class ID: 25, Class Name: umbrella
Class ID: 26, Class Name: handbag
Class ID: 27, Class Name: tie
Class ID: 28, Class Name: suitcase
Class ID: 29, Class Name: frisbee
Class ID:

In [81]:
import supervision as sv
from ultralytics import YOLO
import os
import json
import cv2  # OpenCV for image saving
import numpy as np


TARGET_VIDEO_PATH = 'output_video.mp4'
FRAME_SAVE_DIR = 'frames/'  # Directory to save frames
FRAME_DATA_PATH = 'frame_data.json'  # JSON file to save frame data

# Initialize box annotator for drawing bounding boxes
box_annotator = sv.BoxAnnotator(
    thickness=4,
    text_thickness=4,
    text_scale=2
)

# Open video info and frame generator
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
generator = sv.video.get_video_frames_generator(SOURCE_VIDEO_PATH)

# Initialize sequential ID mapping
id_counter = 1
id_map = {}  # Maps tracker_id to a sequential ID
frame_data_list = []  # To store frame data

# Create directory for saving frames
os.makedirs(FRAME_SAVE_DIR, exist_ok=True)

# Open output video stream
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:  
    # Iterate through each frame in the video and track objects
    for frame_number, result in enumerate(
        YOLO('yolov8s.pt').track(
            source=SOURCE_VIDEO_PATH, 
            tracker='bytetrack.yaml', 
            show=False, 
            stream=True, 
            agnostic_nms=True, 
            persist=True
        )
    ):
        # Extract frame and detections
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)
        # Filter detections to only include people (class_id = 0 in COCO)
        PEOPLE_CLASS_ID = 0
        filtered_detections = [
            (bbox, confidence, class_id, tracker_id)
            for bbox, confidence, class_id, tracker_id in detections
            if class_id == PEOPLE_CLASS_ID
        ]
        
        if filtered_detections:
            people_detections = sv.Detections(
                xyxy=np.array([bbox for bbox, _, _, _ in filtered_detections]),
                confidence=np.array([confidence for _, confidence, _, _ in filtered_detections]),
                class_id=np.array([class_id for _, _, class_id, _ in filtered_detections]),
                tracker_id=np.array([tracker_id for _, _, _, tracker_id in filtered_detections]) if detections.tracker_id is not None else None
            )
        else:
            # Create empty detections if no people found
            people_detections = sv.Detections(
                xyxy=np.empty((0, 4)),
                confidence=np.empty((0,)),
                class_id=np.empty((0,), dtype=int),
                tracker_id=None
            )
        
        # Use people_detections instead of detections
        detections = people_detections

        # Handle object IDs (tracker IDs)
        if result.boxes.id is not None:
            for tracker_id in result.boxes.id.cpu().numpy().astype(int):
                # Assign sequential ID if not already assigned
                if tracker_id not in id_map:
                    id_map[tracker_id] = id_counter
                    id_counter += 1

            # Update detections with new sequential IDs
            detections.tracker_id = [id_map[tracker_id] for tracker_id in result.boxes.id.cpu().numpy().astype(int)]

        # Define labels for each detection
        labels = [
            f"ID: {tracker_id} | {model.model.names[class_id]} {confidence:0.2f}"
            for bbox, confidence, class_id, tracker_id in detections
        ]

        # Annotate the frame
        frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)

        # Save current frame to disk
        frame_path = os.path.join(FRAME_SAVE_DIR, f"frame_{frame_number:04d}.jpg")
        cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # Convert RGB to BGR for OpenCV

        # Collect frame data for JSON
        frame_data = {
            "frame_number": frame_number,
            "detections": [
                {
                    "tracker_id": int(tracker_id) if tracker_id is not None else None,  # Handle None tracker_id
                    "class_id": int(class_id),
                    "confidence": float(confidence),
                    "bbox": [float(coord) for coord in bbox]
                }
                for bbox, confidence, class_id, tracker_id in detections
            ]
        }
        frame_data_list.append(frame_data)
        print("Frame data : " + str(frame_data))


        # Write annotated frame to the output video
        sink.write_frame(frame)

    print("Vehicle tracking completed and output video saved.")

# Save frame data to a JSON file
with open(FRAME_DATA_PATH, 'w') as json_file:
    json.dump(frame_data_list, json_file, indent=4)

print(f"Frames saved to '{FRAME_SAVE_DIR}' and frame data saved to '{FRAME_DATA_PATH}'.")


video 1/1 (frame 1/400) /kaggle/working/people-counting.mp4: 384x640 1 person, 1 bench, 9.0ms
Frame data : {'frame_number': 0, 'detections': [{'tracker_id': 1, 'class_id': 0, 'confidence': 0.9044070839881897, 'bbox': [41.92584228515625, 56.4471435546875, 148.6084442138672, 363.50634765625]}]}
video 1/1 (frame 2/400) /kaggle/working/people-counting.mp4: 384x640 1 person, 1 bench, 8.3ms
Frame data : {'frame_number': 1, 'detections': [{'tracker_id': 1, 'class_id': 0, 'confidence': 0.9047181010246277, 'bbox': [42.29210662841797, 56.28046417236328, 148.61566162109375, 362.1519775390625]}]}
video 1/1 (frame 3/400) /kaggle/working/people-counting.mp4: 384x640 1 person, 2 benchs, 8.3ms
Frame data : {'frame_number': 2, 'detections': [{'tracker_id': 1, 'class_id': 0, 'confidence': 0.9029794931411743, 'bbox': [43.26971435546875, 55.400264739990234, 149.5592498779297, 360.6907653808594]}]}
video 1/1 (frame 4/400) /kaggle/working/people-counting.mp4: 384x640 1 person, 2 benchs, 8.4ms
Frame data : 

In [82]:
from IPython.display import FileLink
FileLink(r'output_video.mp4')

/kaggle/working/output_video.mp4